# Classification wine

In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.set_option('max_columns', None)

In [3]:
df_wines = pd.read_csv('df_wines_toClassification.csv', usecols=['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar','chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'type'])
df_wines.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,type
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,White
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,White
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,White
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,White
4,6.2,0.32,0.16,7.0,0.045,30.0,136.0,0.9949,3.18,0.47,White


Segun la conclusion del analisis de los datos, para la clasificacion del vino segun su tipo, excluimos el alcohol y quality.

In [4]:
df_wines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5302 entries, 0 to 5301
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         5302 non-null   float64
 1   volatile acidity      5302 non-null   float64
 2   citric acid           5302 non-null   float64
 3   residual sugar        5302 non-null   float64
 4   chlorides             5302 non-null   float64
 5   free sulfur dioxide   5302 non-null   float64
 6   total sulfur dioxide  5302 non-null   float64
 7   density               5302 non-null   float64
 8   pH                    5302 non-null   float64
 9   sulphates             5302 non-null   float64
 10  type                  5302 non-null   object 
dtypes: float64(10), object(1)
memory usage: 455.8+ KB


#### Cambiaremos los valores de type, 0-> Red, 1 -> White

In [5]:
from sklearn.preprocessing import LabelEncoder

In [6]:
le = LabelEncoder()
df_wines['type_encode'] = le.fit_transform(df_wines['type'])

In [7]:
df_wines[['type_encode','type']].head(2)

,type_encode,type
0,1,White
1,1,White


In [8]:
df_wines[['type_encode','type']].tail(2)

,type_encode,type
5300,0,Red
5301,0,Red


In [9]:
df_wines.type_encode.unique()

array([1, 0])

In [10]:
from sklearn.preprocessing import StandardScaler

In [11]:
scaler = StandardScaler()
df_wines[['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar','chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density','pH', 'sulphates']] = scaler.fit_transform(df_wines[['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar','chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density','pH', 'sulphates']])

In [12]:
df_wines

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,type,type_encode
0,-0.159701,-0.440838,0.292387,3.553645,-0.324804,0.863307,0.998873,2.238849,-1.405957,-0.565671,White,1
1,-0.693989,-0.261561,0.154343,-0.778445,-0.211410,-0.917994,0.323077,-0.180141,0.467977,-0.290775,White,1
2,0.679896,-0.381079,0.568475,0.423653,-0.183062,0.001387,-0.299366,0.199986,0.218119,-0.634395,White,1
3,-0.007047,-0.679873,0.016299,0.786551,0.043726,0.978230,1.283418,0.372771,-0.219132,-0.909291,White,1
4,-0.770316,-0.142043,-1.088053,0.446335,-0.324804,0.001387,0.394213,0.130872,-0.281597,-0.428223,White,1
...,...,...,...,...,...,...,...,...,...,...,...,...
5297,-0.312355,1.650725,-1.640229,-0.710401,0.327210,-0.113535,-1.348628,0.687240,1.217550,1.977120,Red,0
5298,-0.770316,1.531207,-1.640229,-0.687720,0.950876,0.116310,-1.241923,0.130872,1.404944,0.327742,Red,0
5299,-0.999297,1.232412,-1.502185,-0.642358,0.157119,0.518539,-1.117434,0.206897,1.842195,1.564775,Red,0
5300,-0.999297,1.800122,-1.364141,-0.687720,0.525649,0.116310,-1.241923,0.327847,2.154517,1.221155,Red,0


Con los datos estandarizados, procedemos a implementar algoritmos de clasificacion pero antes:

In [13]:
df_wines.shape

(5302, 12)

In [14]:
df_wines_red = df_wines[df_wines['type']== 'Red']

In [15]:
df_wines[df_wines['type']== 'White'].shape

(3951, 12)

La idea es utilizar un dataset de entrenamiento balanceado segun las clases del tipo de vino. Por esa razon aleatoriamente escogeremos 1351 datos de vino blanco igualando la cantidad de datos que tiene el vino rojo.

In [16]:
df_wines_white = df_wines[df_wines['type']== 'White'].sample(1351)


In [17]:
df_wines_white.shape

(1351, 12)

In [18]:
df_wines_class = pd.concat([df_wines_white,df_wines_red], ignore_index= True)
df_wines_class.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,type,type_encode
0,-1.533585,0.096993,-2.054361,-0.778445,-0.721682,-0.343381,-0.174877,-1.700649,1.342479,-1.252912,White,1
1,-0.083374,-0.679873,0.499453,1.965968,0.043726,-0.228458,1.034441,1.046633,-2.030601,-0.496947,White,1
2,-0.236028,0.336028,-1.019031,1.784519,-0.664985,1.667766,1.283418,1.043177,-0.906241,-0.290775,White,1
3,-1.228278,-0.500597,-0.950009,-0.823807,-0.636637,-0.688149,0.376429,-0.961129,0.592906,-1.252912,White,1
4,2.359088,-0.560355,1.120650,-0.642358,-0.409849,-0.113535,0.892168,1.202139,-0.219132,0.396466,White,1


In [19]:
df_wines_class = df_wines_class.sample(frac=1)

In [20]:
df_wines_class.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,type,type_encode
1846,2.053780,0.933618,-0.535877,-0.619677,1.319405,-1.377685,-1.775446,1.133025,0.717835,-0.084602,Red,0
463,0.145607,-0.022525,0.706518,-0.891850,-0.664985,-0.745610,1.016657,-0.974952,-0.656383,-0.015878,White,1
1358,0.450915,1.411689,-2.054361,-0.687720,0.468952,-1.205301,-1.704309,0.787455,0.842764,0.259018,Red,0
1546,-0.312355,1.590966,-1.916317,-0.801126,0.015377,-1.435146,-1.846582,0.251822,1.217550,0.465190,Red,0
2038,1.061530,1.232412,-1.571207,-0.392866,0.327210,-1.262762,-1.722093,0.977519,0.030726,-0.634395,Red,0


In [21]:
from sklearn.model_selection import train_test_split

In [22]:
X = df_wines_class.drop(['type','type_encode'], axis=1)
y = df_wines_class['type_encode']

In [23]:
x_train, x_test, y_train, y_test = train_test_split(X,y,test_size=0.2, stratify=y)

In [24]:
from sklearn.metrics import accuracy_score

In [25]:
from sklearn.metrics import confusion_matrix

In [26]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [27]:
clf1 = KNeighborsClassifier()
clf2 = LogisticRegression()
clf3 = SVC()
clf4 = DecisionTreeClassifier()
clf5 = RandomForestClassifier()

In [28]:
clf1.fit(x_train,y_train)
y_pred_clf1 = clf1.predict(x_test)
print("Metrica: ", accuracy_score(y_pred_clf1,y_test))

Metrica:  0.9907578558225508


In [29]:
confusion_matrix(y_test,y_pred_clf1)

array([[268,   3],
       [  2, 268]], dtype=int64)

In [30]:
clf2.fit(x_train,y_train)
y_pred_clf2 = clf2.predict(x_test)
print("Metrica: ", accuracy_score(y_pred_clf2,y_test))

Metrica:  0.988909426987061


In [31]:
confusion_matrix(y_test,y_pred_clf2)

array([[268,   3],
       [  3, 267]], dtype=int64)

In [32]:
clf3.fit(x_train,y_train)
y_pred_clf3 = clf3.predict(x_test)
print("Metrica: ", accuracy_score(y_pred_clf3,y_test))

Metrica:  0.9926062846580407


In [33]:
confusion_matrix(y_test,y_pred_clf3)

array([[268,   3],
       [  1, 269]], dtype=int64)

In [34]:
clf4.fit(x_train,y_train)
y_pred_clf4 = clf4.predict(x_test)
print("Metrica: ", accuracy_score(y_pred_clf4,y_test))

Metrica:  0.977818853974122


In [35]:
confusion_matrix(y_test,y_pred_clf4)

array([[265,   6],
       [  6, 264]], dtype=int64)

In [36]:
clf5.fit(x_train,y_train)
y_pred_clf5 = clf5.predict(x_test)
print("Metrica: ", accuracy_score(y_pred_clf5,y_test))

Metrica:  0.988909426987061


In [37]:
confusion_matrix(y_test,y_pred_clf5)

array([[266,   5],
       [  1, 269]], dtype=int64)

## PRUEBA FINAL

In [38]:
r_5 = df_wines[df_wines['type'] == 'Red'].sample(5)
w_5 = df_wines[df_wines['type'] =='White'].sample(5)

In [39]:
r_5

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,type,type_encode
4801,0.756223,0.515305,-0.190767,-0.778445,0.695740,-0.170997,-1.224139,0.521366,0.155655,0.052846,Red,0
5298,-0.770316,1.531207,-1.640229,-0.687720,0.950876,0.116310,-1.241923,0.130872,1.404944,0.327742,Red,0
4457,-1.686239,0.455546,-0.535877,-0.687720,0.100423,-0.630687,-1.135218,-0.974952,3.091484,1.427327,Red,0
4704,0.298261,2.218435,-2.192405,-0.665039,0.327210,0.001387,-1.401980,0.279467,1.592337,0.808811,Red,0
4026,0.450915,0.515305,2.639134,-0.710401,11.553191,-0.458303,-0.832889,0.994797,-0.593919,5.138427,Red,0


In [40]:
rw_10 = pd.concat([r_5,w_5],ignore_index=True)
rw_10

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,type,type_encode
0,0.756223,0.515305,-0.190767,-0.778445,0.695740,-0.170997,-1.224139,0.521366,0.155655,0.052846,Red,0
1,-0.770316,1.531207,-1.640229,-0.687720,0.950876,0.116310,-1.241923,0.130872,1.404944,0.327742,Red,0
2,-1.686239,0.455546,-0.535877,-0.687720,0.100423,-0.630687,-1.135218,-0.974952,3.091484,1.427327,Red,0
3,0.298261,2.218435,-2.192405,-0.665039,0.327210,0.001387,-1.401980,0.279467,1.592337,0.808811,Red,0
4,0.450915,0.515305,2.639134,-0.710401,11.553191,-0.458303,-0.832889,0.994797,-0.593919,5.138427,Red,0
5,0.069280,-0.679873,-0.535877,-0.937212,-0.721682,-0.056074,-0.494991,-1.818143,-2.030601,-1.046739,White,1
6,-0.617662,0.037234,-0.259789,-0.778445,-0.551591,0.058849,-0.014821,-2.326130,-0.656383,-0.909291,White,1
7,0.145607,-0.799391,3.329354,1.648433,-0.523243,2.702070,1.230066,1.126114,-1.718279,-0.359499,White,1
8,-0.159701,-0.142043,-0.190767,-0.029968,-0.579940,0.633462,0.643191,-0.981863,0.967692,-0.703119,White,1
9,-0.388681,-0.620114,0.223365,1.829881,-0.183062,1.955073,1.621316,0.856569,-0.468990,-0.222051,White,1


In [41]:
rw_10 = rw_10.sample(frac=1)

In [42]:
rw_10

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,type,type_encode
8,-0.159701,-0.142043,-0.190767,-0.029968,-0.579940,0.633462,0.643191,-0.981863,0.967692,-0.703119,White,1
7,0.145607,-0.799391,3.329354,1.648433,-0.523243,2.702070,1.230066,1.126114,-1.718279,-0.359499,White,1
1,-0.770316,1.531207,-1.640229,-0.687720,0.950876,0.116310,-1.241923,0.130872,1.404944,0.327742,Red,0
6,-0.617662,0.037234,-0.259789,-0.778445,-0.551591,0.058849,-0.014821,-2.326130,-0.656383,-0.909291,White,1
5,0.069280,-0.679873,-0.535877,-0.937212,-0.721682,-0.056074,-0.494991,-1.818143,-2.030601,-1.046739,White,1
0,0.756223,0.515305,-0.190767,-0.778445,0.695740,-0.170997,-1.224139,0.521366,0.155655,0.052846,Red,0
2,-1.686239,0.455546,-0.535877,-0.687720,0.100423,-0.630687,-1.135218,-0.974952,3.091484,1.427327,Red,0
3,0.298261,2.218435,-2.192405,-0.665039,0.327210,0.001387,-1.401980,0.279467,1.592337,0.808811,Red,0
4,0.450915,0.515305,2.639134,-0.710401,11.553191,-0.458303,-0.832889,0.994797,-0.593919,5.138427,Red,0
9,-0.388681,-0.620114,0.223365,1.829881,-0.183062,1.955073,1.621316,0.856569,-0.468990,-0.222051,White,1


In [43]:
rw_10_x = rw_10.drop(['type', 'type_encode'],axis=1)
rw_10_Y = rw_10['type_encode']

In [44]:
rw_10_Y

8    1
7    1
1    0
6    1
5    1
0    0
2    0
3    0
4    0
9    1
Name: type_encode, dtype: int32

In [45]:
y_pred_rw10 = clf3.predict(rw_10_x)

In [46]:
y_prueba = pd.Series(y_pred_rw10)

In [47]:
accuracy_score(rw_10_Y,y_prueba)

1.0

In [48]:
y_pred_rw10 = clf2.predict(rw_10_x)

In [49]:
y_prueba = pd.Series(y_pred_rw10)
accuracy_score(rw_10_Y,y_prueba)

1.0

In [50]:
y_pred_rw10 = clf1.predict(rw_10_x)
y_prueba = pd.Series(y_pred_rw10)
accuracy_score(rw_10_Y,y_prueba)

1.0

In [51]:
y_pred_rw10 = clf4.predict(rw_10_x)
y_prueba = pd.Series(y_pred_rw10)
accuracy_score(rw_10_Y,y_prueba)

1.0

In [52]:
y_pred_rw10 = clf5.predict(rw_10_x)
y_prueba = pd.Series(y_pred_rw10)
accuracy_score(rw_10_Y,y_prueba)

1.0

#### Conclusion

Se probaron diferentes algoritmos de clasificacion para encontrar el tipo de vino segun sus caracteristicas. En orden de mejor a menor encontramos:

* SVM
* Logistic Regression
* KNN
* Random Forest 
* Decision Tree

De igual forma en la prueba con nuevos datos aleatorios, todos clasificaron bien.

#### Recuerdan los dos pares de datos que tenian los mismos datos pero uno para vino rojo y otro para vino blanco. Veamos como clasifican nuestros algoritmos cada uno

In [57]:
datos1 = [6.7,0.28,0.28,2.4,0.012,36.0,100.0,0.99064,3.26,0.39]
datos2 = [5.9,0.19,0.21,1.7,0.045,57.0,135.0,0.99341,3.32,0.44]

In [59]:
datos_duplicados = pd.DataFrame([datos1,datos2], columns=['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar','chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density','pH', 'sulphates'])

In [60]:
datos_duplicados

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates
0,6.7,0.28,0.28,2.4,0.012,36.0,100.0,0.99064,3.26,0.39
1,5.9,0.19,0.21,1.7,0.045,57.0,135.0,0.99341,3.32,0.44


### Escalamos los datos

In [61]:
datosDupli_scaler = scaler.fit_transform(datos_duplicados)


In [64]:
y_pred = clf1.predict(datosDupli_scaler)
y_pred

array([1, 1])

In [65]:
y_pred = clf2.predict(datosDupli_scaler)
y_pred

array([1, 1])

In [66]:
y_pred = clf3.predict(datosDupli_scaler)
y_pred

array([1, 1])

In [67]:
y_pred = clf4.predict(datosDupli_scaler)
y_pred

array([1, 1])

In [68]:
y_pred = clf5.predict(datosDupli_scaler)
y_pred

array([1, 1])

## Finalmente, vemos que todos los algoritmos clasifican de Vino blanco los datos que sabemos tambien pueden ser vino Rojo.